In [1]:
import zipfile
import os

**Preparar carpeta de imagenes**

In [2]:
!wget --no-check-certificate \
    "https://github.com/arbeyvillegas/IntrDeepLearning/raw/main/Entrega-02/data/brain_tumor_data.zip" \
    -O "/tmp/brain_tumor_data.zip"

zip_ref = zipfile.ZipFile('/tmp/brain_tumor_data.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-11-19 16:23:14--  https://github.com/arbeyvillegas/IntrDeepLearning/raw/main/Entrega-02/data/brain_tumor_data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/arbeyvillegas/IntrDeepLearning/main/Entrega-02/data/brain_tumor_data.zip [following]
--2023-11-19 16:23:14--  https://raw.githubusercontent.com/arbeyvillegas/IntrDeepLearning/main/Entrega-02/data/brain_tumor_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91021706 (87M) [application/zip]
Saving to: ‘/tmp/brain_tumor_data.zip’

/tmp/brain_tumor_da 100%[===================>]  86.80M   302MB/s    in 0.3s    

2023-11-19 16:

In [3]:
carpeta_training = "/tmp/brain_tumor_data/Training"
carpeta_testing = "/tmp/brain_tumor_data/Testing"

**Definir el modelo CNN**

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import time
import numpy as np
import keras.optimizers
from sklearn.metrics import classification_report

In [5]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [6]:
batch_size = 32
nb_classes = 4
nb_epochs = 10
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3
img_size = 224

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(img_size,img_size,img_channel)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(4,  activation=tf.nn.softmax)
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(carpeta_training, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(carpeta_testing, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')




Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [8]:
steps_per_epoch = 2870//batch_size

validation_steps = 394//batch_size

In [9]:
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=nb_epochs, validation_data=validation_generator, validation_steps=validation_steps, callbacks=[tensorboard])

<ipython-input-9-b974c5a2df44>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=nb_epochs, validation_data=validation_generator, validation_steps=validation_steps, callbacks=[tensorboard])


Epoch 1/10
89/89 [==============================] - 216s 2s/step - loss: 1.0362 - accuracy: 0.5617 - val_loss: 2.3957 - val_accuracy: 0.3984
Epoch 2/10
89/89 [==============================] - 217s 2s/step - loss: 0.7264 - accuracy: 0.6910 - val_loss: 2.2798 - val_accuracy: 0.4219
Epoch 3/10
89/89 [==============================] - 214s 2s/step - loss: 0.6424 - accuracy: 0.7283 - val_loss: 3.1820 - val_accuracy: 0.3724
Epoch 4/10
89/89 [==============================] - 216s 2s/step - loss: 0.6007 - accuracy: 0.7474 - val_loss: 2.5498 - val_accuracy: 0.4766
Epoch 5/10
89/89 [==============================] - 217s 2s/step - loss: 0.5191 - accuracy: 0.7833 - val_loss: 2.6699 - val_accuracy: 0.4427
Epoch 6/10
89/89 [==============================] - 216s 2s/step - loss: 0.4802 - accuracy: 0.7970 - val_loss: 2.7380 - val_accuracy: 0.5052
Epoch 7/10
89/89 [==============================] - 212s 2s/step - loss: 0.4623 - accuracy: 0.7988 - val_loss: 2.9525 - val_accuracy: 0.4792
Epoch 8/10
89

In [10]:
model.save("cnn_augmented_data_generator_10-epochs_0.56-accuracy.keras")